In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import requests


C:\Users\connm\AppData\Local\Temp/ipykernel_6456/2808074499.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\connm\AppData\Local\Temp/ipykernel_6456/2808074499.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
spacex_df = pd.read_csv(url)

In [3]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
# Read the airline data into pandas dataframe

max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                  {'label': 'All Sites', 'value': 'All Sites'},
                                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                              ],
                                              value='All Sites',
                                              placeholder="Select a site",
                                              searchable=True
                                              ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                1000: '1000',
                                                2000: '2000',
                                                3000: '3000',
                                                4000: '4000',
                                                5000: '5000',
                                                6000: '6000',
                                                7000: '7000',
                                                8000: '8000',
                                                9000: '9000',
                                                10000: '10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'All Sites':
        fig = px.pie(spacex_df, values= 'class',
        names='Launch Site',
        title='All Sites')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_site = filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_site, values='class count',
        names='class',
        title=f'Successful Launches for {entered_site}')
        return fig
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, slider):
    df_scatter = spacex_df[(spacex_df['Payload Mass (kg)'] > slider[0]) & (spacex_df['Payload Mass (kg)'] < slider[1])]
    if entered_site == 'All Sites':
        fig_scat = px.scatter(df_scatter, x='Payload Mass (kg)', y='class',
                   color='Booster Version Category',
                   hover_data=['Booster Version'],
                   title='Success rate by payload size for all sites')
        return fig_scat
    else:
        df_scatter_site = df_scatter[df_scatter['Launch Site'] == entered_site]
        fig_scat = px.scatter(df_scatter_site, x='Payload Mass (kg)', y='class',
                   color='Booster Version Category',
                   hover_data=['Booster Version'],
                   title= f'Success by payload Size for {entered_site}')
        return fig_scat

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2022 20:35:49] "POST /_dash-